In [1]:
! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
import setup_google_colab
# please, uncomment the week you're working on
# setup_google_colab.setup_week1()  
# setup_google_colab.setup_week2()
# setup_google_colab.setup_week3()
# setup_google_colab.setup_week4()
#setup_google_colab.setup_project()
setup_google_colab.setup_honor()

--2021-09-18 18:21:11--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1939 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.89K  --.-KB/s    in 0s      

2021-09-18 18:21:11 (28.9 MB/s) - ‘setup_google_colab.py’ saved [1939/1939]



In [2]:
%tensorflow_version 2.x
import tensorflow as tf

In [3]:
!pip install compress-pickle

In [4]:
from compress_pickle import load,dump

In [5]:
!cat /content/download_cornell.sh

#!/bin/bash

mkdir -p data/cornell
cd data/cornell
wget https://github.com/Conchylicultor/DeepQA/raw/master/data/cornell/movie_conversations.txt
wget https://github.com/Conchylicultor/DeepQA/raw/master/data/cornell/movie_lines.txt


In [5]:
!sudo sh ./download_cornell.sh

--2021-09-18 18:22:45--  https://github.com/Conchylicultor/DeepQA/raw/master/data/cornell/movie_conversations.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Conchylicultor/DeepQA/master/data/cornell/movie_conversations.txt [following]
--2021-09-18 18:22:46--  https://raw.githubusercontent.com/Conchylicultor/DeepQA/master/data/cornell/movie_conversations.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6760930 (6.4M) [text/plain]
Saving to: ‘movie_conversations.txt’

movie_conversations 100%[===================>]   6.45M  --.-KB/s    in 0.09s   

2021-09-18 18:22:46 (72.0 MB/s) - ‘movie_c

In [7]:
!mkdir -p data/opensubs; cd /content/data/opensubs; wget -O en.tar.gz http://opus.lingfil.uu.se/download.php?f=OpenSubtitles/en.tar.gz; unzip en.tar.gz; rm en.tar.gz

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: OpenSubtitles/xml/en/Adventure/2003/2144_147426_201372_secondhand_lions.xml  
  inflating: OpenSubtitles/xml/en/Adventure/2003/1825_126646_174379_tokyo_godfathers.xml  
  inflating: OpenSubtitles/xml/en/Adventure/2003/2144_61093_78347_secondhand_lions.info  
  inflating: OpenSubtitles/xml/en/Adventure/2003/2144_61093_78347_secondhand_lions.xml  
  inflating: OpenSubtitles/xml/en/Adventure/2003/2144_61043_78289_secondhand_lions.xml  
  inflating: OpenSubtitles/xml/en/Adventure/2003/1864_69243_89778_peter_pan.xml  
  inflating: OpenSubtitles/xml/en/Adventure/2003/2144_147426_201372_secondhand_lions.info  
  inflating: OpenSubtitles/xml/en/Adventure/2003/3084_125320_172655_sinbad_legend_of_the_seven_seas.xml  
  inflating: OpenSubtitles/xml/en/Adventure/2003/745_84434_112858_riverworld.xml  
  inflating: OpenSubtitles/xml/en/Adventure/2003/452_68791_89169_timeline.xml  
  inflating: OpenSubtitles/xml/en/Advent

In [6]:
# Copyright 2015 Conchylicultor. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import ast
import os
import random
import re
from time import time

import nltk
from tqdm import tqdm

"""
Load the cornell movie dialog corpus.
Available from here:
http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html
"""

class CornellData:
    """
    """

    def __init__(self, dirName):
        """
        Args:
            dirName (string): directory where to load the corpus
        """
        self.lines = {}
        self.conversations = []

        MOVIE_LINES_FIELDS = ["lineID","characterID","movieID","character","text"]
        MOVIE_CONVERSATIONS_FIELDS = ["character1ID","character2ID","movieID","utteranceIDs"]

        self.lines = self.loadLines(os.path.join(dirName, "movie_lines.txt"), MOVIE_LINES_FIELDS)
        self.conversations = self.loadConversations(os.path.join(dirName, "movie_conversations.txt"), MOVIE_CONVERSATIONS_FIELDS)

        # TODO: Cleaner program (merge copy-paste) !!

    def loadLines(self, fileName, fields):
        """
        Args:
            fileName (str): file to load
            field (set<str>): fields to extract
        Return:
            dict<dict<str>>: the extracted fields for each line
        """
        lines = {}

        with open(fileName, 'r', encoding='iso-8859-1') as f:  # TODO: Solve Iso encoding pb !
            for line in f:
                values = line.split(" +++$+++ ")

                # Extract fields
                lineObj = {}
                for i, field in enumerate(fields):
                    lineObj[field] = values[i]

                lines[lineObj['lineID']] = lineObj

        return lines

    def loadConversations(self, fileName, fields):
        """
        Args:
            fileName (str): file to load
            field (set<str>): fields to extract
        Return:
            list<dict<str>>: the extracted fields for each line
        """
        conversations = []

        with open(fileName, 'r', encoding='iso-8859-1') as f:  # TODO: Solve Iso encoding pb !
            for line in f:
                values = line.split(" +++$+++ ")

                # Extract fields
                convObj = {}
                for i, field in enumerate(fields):
                    convObj[field] = values[i]

                # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
                lineIds = ast.literal_eval(convObj["utteranceIDs"])

                # Reassemble lines
                convObj["lines"] = []
                for lineId in lineIds:
                    convObj["lines"].append(self.lines[lineId])

                conversations.append(convObj)

        return conversations

    def getConversations(self):
        return self.conversations

In [7]:
import xml.etree.ElementTree as ET
import datetime
import os
import sys
import json
import re
import pprint

from gzip import GzipFile

"""
Load the opensubtitles dialog corpus.
"""

class OpensubsData:
    """
    """

    def __init__(self, dirName):
        """
        Args:
            dirName (string): directory where to load the corpus
        """

        # Hack this to filter on subset of Opensubtitles
        # dirName = "%s/en/Action" % dirName

        print("Loading OpenSubtitles conversations in %s." % dirName)
        self.conversations = []
        self.tag_re = re.compile(r'(<!--.*?-->|<[^>]*>)')
        self.conversations = self.loadConversations(dirName)

    def loadConversations(self, dirName):
        """
        Args:
            dirName (str): folder to load
        Return:
            array(question, answer): the extracted QA pairs
        """
        conversations = []
        dirList = self.filesInDir(dirName)
        print(dirList)
        for filepath in tqdm(dirList):
              #print(filepath)
              #if filepath.endswith('gz'):
          try:
            doc = self.getXML(filepath)
            conversations.extend(self.genList(doc))
          except ValueError:
            tqdm.write("Skipping file %s with errors." % filepath)
          except:
            print("Unexpected error:")
                      #raise
        return conversations

    def getConversations(self):
        return self.conversations

    def genList(self, tree):
        root = tree.getroot()

        timeFormat = '%H:%M:%S'
        maxDelta = datetime.timedelta(seconds=1)

        startTime = datetime.datetime.min
        strbuf = ''
        sentList = []

        for child in root:
            for elem in child:
                if elem.tag == 'time':
                    elemID = elem.attrib['id']
                    elemVal = elem.attrib['value'][:-4]
                    if elemID[-1] == 'S':
                        startTime = datetime.datetime.strptime(elemVal, timeFormat)
                    else:
                        sentList.append((strbuf.strip(), startTime, datetime.datetime.strptime(elemVal, timeFormat)))
                        strbuf = ''
                else:
                    try:
                        strbuf = strbuf + " " + elem.text
                    except:
                        pass

        conversations = []
        for idx in range(0, len(sentList) - 1):
            cur = sentList[idx]
            nxt = sentList[idx + 1]
            if nxt[1] - cur[2] <= maxDelta and cur and nxt:
                tmp = {}
                tmp["lines"] = []
                tmp["lines"].append(self.getLine(cur[0]))
                tmp["lines"].append(self.getLine(nxt[0]))
                if self.filter(tmp):
                    conversations.append(tmp)

        return conversations

    def getLine(self, sentence):
        line = {}
        line["text"] = self.tag_re.sub('', sentence).replace('\\\'','\'').strip().lower()
        return line

    def filter(self, lines):
        # Use the followint to customize filtering of QA pairs
        #
        # startwords = ("what", "how", "when", "why", "where", "do", "did", "is", "are", "can", "could", "would", "will")
        # question = lines["lines"][0]["text"]
        # if not question.endswith('?'):
        #     return False
        # if not question.split(' ')[0] in startwords:
        #     return False
        #
        return True

    def getXML(self, filepath):
      fext = os.path.splitext(filepath)[1]
      if fext == '.gz':
          tmp = GzipFile(filename=filepath)
          return ET.parse(tmp)
      else:
          return ET.parse(filepath)

    def filesInDir(self, dirname):
        result = []
        for dirpath, dirs, files in os.walk(dirname):
            for filename in files:
                fname = os.path.join(dirpath, filename)
                result.append(fname)
        return result

def extractText(line, fast_preprocessing=True):
    if fast_preprocessing:
        GOOD_SYMBOLS_RE = re.compile('[^0-9a-z ]')
        REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;#+_]')
        REPLACE_SEVERAL_SPACES = re.compile('\s+')

        line = line.lower()
        line = REPLACE_BY_SPACE_RE.sub(' ', line)
        line = GOOD_SYMBOLS_RE.sub('', line)
        line = REPLACE_SEVERAL_SPACES.sub(' ', line)
        return line.strip()
    else:
        return nltk.word_tokenize(line)


def splitConversations(conversations, max_len=20, fast_preprocessing=True):
    data = []
    for i, conversation in enumerate(tqdm(conversations)):
        lines = conversation['lines']
        for i in range(len(lines) - 1):
            request = extractText(lines[i]['text'])
            reply = extractText(lines[i + 1]['text'])
            if 0 < len(request) <= max_len and 0 < len(reply) <= max_len:
                data += [(request, reply)]
    return data


def readCornellData(path, max_len=20, fast_preprocessing=True):
    dataset = CornellData(path)
    conversations = dataset.getConversations()
    return splitConversations(conversations, max_len=max_len, fast_preprocessing=fast_preprocessing)


def readOpensubsData(path, max_len=20, fast_preprocessing=True):
    dataset = OpensubsData(path)
    conversations = dataset.getConversations()
    return splitConversations(conversations, max_len=max_len, fast_preprocessing=fast_preprocessing)


In [10]:
data_opensub = readOpensubsData('/content/data/opensubs')

Loading OpenSubtitles conversations in /content/data/opensubs.
['/content/data/opensubs/LICENSE', '/content/data/opensubs/INFO', '/content/data/opensubs/README', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1999/1898_92588_125113_kaosu.info', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1999/1898_92588_125113_kaosu.xml', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1967/5367_89066_119891_in_the_heat_of_the_night.info', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1967/5367_89066_119891_in_the_heat_of_the_night.xml', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1972/1196_57983_73868_the_godfather.info', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1972/1196_57983_73868_the_godfather.xml', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1930/8606_102149_139882_little_caesar.info', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1930/8606_102149_139882_little_caesar.xml', '/content/data/opensubs/OpenSubtitles/xml/en/Crime/1996/3872_57386_73075_far

  0%|          | 0/4637 [00:00<?, ?it/s]

Unexpected error:
Unexpected error:
Unexpected error:
Unexpected error:
Unexpected error:


  0%|          | 7/4637 [00:00<03:05, 25.01it/s]

Unexpected error:
Unexpected error:


...

100%|█████████▉| 4625/4637 [03:34<00:00, 43.09it/s]

Unexpected error:
Unexpected error:
Unexpected error:
Unexpected error:
Unexpected error:
Unexpected error:
Unexpected error:
Unexpected error:


100%|██████████| 4637/4637 [03:35<00:00, 21.55it/s]


Unexpected error:
Unexpected error:


100%|██████████| 1648080/1648080 [00:26<00:00, 61420.14it/s]


In [11]:
data_opensub[:2]

[('shall i', 'yes'), ('there', 'thanks')]

In [8]:
data = readCornellData('/content/data/cornell')

100%|██████████| 83097/83097 [00:04<00:00, 18825.01it/s]


In [9]:
#data += data_opensub
data[:2]

[('there', 'where'), ('have fun tonight', 'tons')]

In [10]:
from tqdm import tqdm

In [11]:
def add_tokens(data):
  requests=[]
  replys=[]
  for i in tqdm(data):
    request=i[0]
    reply=i[1]
    request=request.rstrip().strip()
    reply=reply.rstrip().strip()
    request="<start> "+request+" <end>"
    reply="<start> "+reply+" <end>"
    request=request.lower()
    reply=reply.lower()
    requests.append(request)
    replys.append(reply)
  return requests, replys

In [12]:
requests,replys=add_tokens(data)

100%|██████████| 24792/24792 [00:00<00:00, 469709.30it/s]


In [ ]:
tokenizer = load('/content/tokenizer', compression="lzma", set_default_extension=False)

In [13]:
def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""
    
    # Train a vectorizer on X_train data.
    # Transform X_train and X_test data.
    
    # Pickle the trained vectorizer to 'vectorizer_path'
    # Don't forget to open the file in writing bytes mode.
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1, 2), token_pattern='(\S+)')
    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_test = tfidf_vectorizer.transform(X_test)
    with open(vectorizer_path, 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    
    return X_train, X_test

In [14]:
from keras.preprocessing.text import Tokenizer

In [15]:
tokenizer=Tokenizer(filters='')

In [16]:
tokenizer.fit_on_texts(requests+replys)

In [17]:
requests_seq = tokenizer.texts_to_sequences(requests)
requests_seq

[[1, 74, 2],
 [1, 58, 378, 251, 2],
 [1, 5, 43, 495, 2],
 [1, 359, 2],
 [1, 66, 31, 2],
 [1, 86, 13, 20, 2],
 [1, 30, 2],
 [1, 18, 161, 2],
 [1, 81, 457, 4814, 2],
 [1, 1379, 3, 188, 2],
 [1, 5, 2],
 [1, 39, 3007, 49, 8, 784, 2],
 [1, 140, 29, 2],
 [1, 3, 496, 12, 53, 2],
 [1, 956, 11, 16, 8, 2],
 [1, 275, 7, 2],
 [1, 3, 9, 241, 235, 2],
 [1, 4815, 2],
 [1, 76, 1846, 829, 2],
 [1, 72, 276, 2],
 [1, 54, 2],
 [1, 66, 3008, 12, 2],
 [1, 33, 9, 2308, 2],
 [1, 81, 13, 3, 254, 2],
 [1, 34, 147, 367, 2],
 [1, 182, 2],
 [1, 14, 2],
 [1, 18, 59, 116, 2],
 [1, 303, 3, 13, 6, 2],
 [1, 81, 2],
 [1, 24, 368, 94, 37, 2],
 [1, 33, 28, 2],
 [1, 209, 24, 68, 2],
 [1, 58, 3, 422, 50, 2],
 [1, 31, 2],
 [1, 54, 4, 13, 2],
 [1, 30, 2],
 [1, 3009, 2],
 [1, 1380, 533, 53, 2],
 [1, 431, 2],
 [1, 88, 3, 39, 1381, 2],
 [1, 142, 2],
 [1, 7, 3, 957, 2],
 [1, 82, 621, 3, 223, 2],
 [1, 4, 183, 13, 28, 2],
 [1, 13, 5, 2],
 [1, 30, 2],
 [1, 3, 2309, 37, 2],
 [1, 4816, 2],
 [1, 37, 33, 35, 194, 2],
 [1, 2310, 9, 120, 

In [18]:
replys_seq = tokenizer.texts_to_sequences(replys)

In [19]:
from keras.preprocessing.sequence import pad_sequences

In [20]:
requests_seq = pad_sequences(requests_seq, maxlen=13, padding='post', truncating='post')

In [21]:
replys_seq = pad_sequences(replys_seq, maxlen=13, padding='post', truncating='post')

In [23]:
replys_seq

array([[   1,   34,    2, ...,    0,    0,    0],
       [   1, 2971,    2, ...,    0,    0,    0],
       [   1,    9,  231, ...,    0,    0,    0],
       ...,
       [   1,  899,    2, ...,    0,    0,    0],
       [   1, 8870,  151, ...,    0,    0,    0],
       [   1,   10,    2, ...,    0,    0,    0]], dtype=int32)

In [22]:
embedding_dim = 200
units = 1024
vocab_size = len(tokenizer.word_index)+1
BATCH_SIZE=256

In [23]:
BUFFER_SIZE = len(requests_seq)
BATCH_SIZE = 256
steps_per_epoch = len(requests_seq)//BATCH_SIZE
embedding_dim = 200
units = 1024
vocab_size = len(tokenizer.word_index)+1
dataset = tf.data.Dataset.from_tensor_slices((requests_seq,replys_seq)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
example_input_batch, example_target_batch = next(iter(dataset))
print(example_input_batch.shape,example_target_batch.shape)

(256, 13) (256, 13)


In [24]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [25]:
encoder = Encoder(vocab_size, embedding_dim, units, BATCH_SIZE)
encoder.batch_sz

256

In [26]:
sample_hidden = encoder.initialize_hidden_state()

In [27]:
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

In [28]:
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))

Encoder output shape: (batch size, sequence length, units) (256, 13, 1024)


In [29]:
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (256, 1024)


In [30]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [31]:
attention_layer = BahdanauAttention(100)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (256, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (256, 13, 1)


In [32]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [33]:
decoder = Decoder(vocab_size, embedding_dim, units, BATCH_SIZE)

In [34]:
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

In [35]:
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (256, 8871)


In [36]:
len(tokenizer.word_index)

8870

In [37]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [38]:
optimizer.get_weights()

[]

In [39]:
import os

checkpoint_dir = "/content/drive/My Drive/COURSERA/ADVANCED ML/chatbot_checkpoint"
'''checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)'''

'checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")\ncheckpoint = tf.train.Checkpoint(optimizer=optimizer,\n                                 encoder=encoder,\n                                 decoder=decoder)'

In [40]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [41]:
def save_optimizer_state(optimizer, save_path, save_name):
    '''
    Save keras.optimizers object state.

    Arguments:
    optimizer --- Optimizer object.
    save_path --- Path to save location.
    save_name --- Name of the .npy file to be created.

    '''

    # Create folder if it does not exists
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # save weights
    np.save(os.path.join(save_path, save_name), optimizer.get_weights())

    return

def load_optimizer_state(optimizer, load_path, load_name, model_train_vars):
    '''
    Loads keras.optimizers object state.

    Arguments:
    optimizer --- Optimizer object to be loaded.
    load_path --- Path to save location.
    load_name --- Name of the .npy file to be read.
    model_train_vars --- List of model variables (obtained using Model.trainable_variables)

    '''

    # Load optimizer weights
    opt_weights = np.load(os.path.join(load_path, load_name)+'.npy', allow_pickle=True)

    # dummy zero gradients
    zero_grads = [tf.zeros_like(w) for w in model_train_vars]
    # save current state of variables
    saved_vars = [tf.identity(w) for w in model_train_vars]

    # Apply gradients which don't do nothing with Adam
    optimizer.apply_gradients(zip(zero_grads, model_train_vars))

    # Reload variables
    [x.assign(y) for x,y in zip(model_train_vars, saved_vars)]

    # Set the weights of the optimizer
    optimizer.set_weights(opt_weights)


    return

In [42]:
import numpy as np


In [43]:
load_optimizer_state(optimizer, checkpoint_dir, 'opt', encoder.trainable_weights + decoder.trainable_weights)

In [44]:
encoder.load_weights(checkpoint_dir + 'enc')

In [45]:
decoder.load_weights(checkpoint_dir + 'dec')

Model: "encoder_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  1774200   
_________________________________________________________________
gru_2 (GRU)                  multiple                  3766272   
Total params: 5,540,472
Trainable params: 5,540,472
Non-trainable params: 0
_________________________________________________________________


In [46]:
import time

EPOCHS = 10 

for epoch in range(EPOCHS):
    e_start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        b_start = time.time()
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        

        if batch % 5 == 0:
            print('Epoch {} Batch {} Loss {:.4f} time {}'.format(epoch + 1,
                                                    batch,
                                                    batch_loss.numpy(), time.time() - b_start))
        if batch % 100 == 0:
            #checkpoint.save(file_prefix = checkpoint_prefix)
            save_optimizer_state(optimizer, checkpoint_dir, 'opt')
            decoder.save(checkpoint_dir + 'dec')
            encoder.save(checkpoint_dir + 'enc')

            
    # saving (checkpoint) the model every 2 epochs
    #if (epoch + 1) % 2 == 0:
    

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - e_start))

Epoch 1 Batch 0 Loss 0.4791 time 29.82347798347473


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


INFO:tensorflow:Assets written to: /content/drive/My Drive/COURSERA/ADVANCED ML/chatbot_checkpointdec/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/COURSERA/ADVANCED ML/chatbot_checkpointdec/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/COURSERA/ADVANCED ML/chatbot_checkpointenc/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/COURSERA/ADVANCED ML/chatbot_checkpointdec/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/COURSERA/ADVANCED ML/chatbot_checkpointdec/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/COURSERA/ADVANCED ML/chatbot_checkpointenc/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/COURSERA/ADVANCED ML/chatbot_checkpointenc/assets


Epoch 10 Batch 5 Loss 0.2256 time 13.24501371383667
Epoch 10 Batch 10 Loss 0.2605 time 13.347048997879028
Epoch 10 Batch 15 Loss 0.2688 time 13.365995168685913
Epoch 10 Batch 20 Loss 0.2655 time 13.353196144104004
Epoch 10 Batch 25 Loss 0.2285 time 13.456006050109863
Epoch 10 Batch 30 Loss 0.2861 time 13.373031854629517
Epoch 10 Batch 35 Loss 0.2822 time 13.287290573120117
Epoch 10 Batch 40 Loss 0.2817 time 13.279614210128784
Epoch 10 Batch 45 Loss 0.2650 time 13.209840297698975
Epoch 10 Batch 50 Loss 0.2760 time 13.359174489974976
Epoch 10 Batch 55 Loss 0.2578 time 13.351295232772827
Epoch 10 Batch 60 Loss 0.2633 time 13.188128471374512
Epoch 10 Batch 65 Loss 0.2807 time 13.329290866851807
Epoch 10 Batch 70 Loss 0.3048 time 13.500045776367188
Epoch 10 Batch 75 Loss 0.2750 time 13.5812828540802
Epoch 10 Batch 80 Loss 0.2921 time 13.081773281097412
Epoch 10 Batch 85 Loss 0.2907 time 13.135976791381836
Epoch 10 Batch 90 Loss 0.2920 time 13.301910161972046
Epoch 10 Batch 95 Loss 0.3065 ti

In [51]:
max_len=13

In [53]:
import string

def evaluate(sentence):
  attention_plot = np.zeros((max_len,max_len))
  sentence=sentence.lower()
  sentence=sentence.rstrip().strip()
  sentence=sentence.translate(str.maketrans('', '', string.punctuation))
  sentence = "<start> "+sentence+" <end>"

  inputs = [tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_len,padding="post",truncating="post")
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

  for t in range(max_len):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += tokenizer.index_word[predicted_id] + ' '
    #print(result)
    if tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [54]:
def chat(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

In [55]:
import numpy as np

In [61]:
test_input = ['hi', 'how are you?', 'what s your name?', 'what do you want?',
              'i love you', 'how do you feel', 'How is the weather today?',
              'why him?', 'are you ok?', 'let s have dinner', ]

for x in test_input:
    chat(x)
    print('\n')

Input: <start> hi <end>
Predicted translation: hi <end> 


Input: <start> how are you <end>
Predicted translation: fine <end> 


Input: <start> what s your name <end>
Predicted translation: ann bishop millaney <end> 


Input: <start> what do you want <end>
Predicted translation: what do you mean <end> 


Input: <start> i love you <end>
Predicted translation: i love you <end> 


Input: <start> how do you feel <end>
Predicted translation: bruised <end> 


Input: <start> how is the weather today <end>
Predicted translation: gosh <end> 


Input: <start> why him <end>
Predicted translation: why not <end> 


Input: <start> are you ok <end>
Predicted translation: im fine <end> 


Input: <start> let s have dinner <end>
Predicted translation: ill get on <end> 


